In [1]:
import torch
import torch.nn.functional as F
import timm
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from robustbench import load_model
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle



/home/iit/anaconda3/envs/DPP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=196, shuffle=False, num_workers=4)
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Files already downloaded and verified


In [3]:
def load_model_normalizer(i):
    if i==1:
        model = load_model(model_name='Standard', dataset='cifar10', threat_model='Linf').to(device)
    else:  
        model = load_model(model_name='Wu2020Adversarial_extra', dataset='cifar10', threat_model='Linf').to(device) 
    normalizer = lambda x: x
    for p in model.parameters():
        p.requires_grad_(False)
    model.eval()
    return model, normalizer

model, normalizer = load_model_normalizer(1)

Downloading...
From (original): https://drive.google.com/uc?id=1-WJWpAZLlmc4gJ8XXNf7IETjnSZzaCNp
From (redirected): https://drive.google.com/uc?id=1-WJWpAZLlmc4gJ8XXNf7IETjnSZzaCNp&confirm=t&uuid=7d5e2bda-1441-442b-aee7-beef62f83804
To: /home/iit/Downloads/BlindSpots Geometry/models/cifar10/Linf/Wu2020Adversarial_extra.pt
100%|██████████| 153M/153M [00:51<00:00, 2.95MB/s] 


In [4]:
# target_classes = [0,1,2,3,4,5,6,7,8,9]
image_number = list(range(190, 200))
target_images = []
to_tensor = transforms.ToTensor()
for i, tc in enumerate(image_number):
    target_images.append(to_tensor(Image.open(f'./data/CIFAR10/cifar10_target_{tc}.png')))
    target_images[-1] = target_images[-1][:3]
target_images = torch.stack(target_images, dim=0)

with torch.no_grad():
    preds = torch.softmax(model(normalizer(target_images.to(device))), dim=-1).cpu().detach().numpy()
    target_classes = np.argmax(preds, axis=1)
    confidence_levels = np.max(preds, axis=1)
    print(target_classes)
    print(confidence_levels)
    target_classes_str = "-".join([str(x) for x in target_classes])


[8 7 8 8 2 5 2 3 5 0]
[0.9999995  0.99975437 0.99999976 0.99999917 0.99998903 0.99953127
 0.999998   0.9998977  0.9999825  0.9999721 ]


## "Running attack on examples..."

In [155]:
def dict_combine(d1, d2):
    for k in d2:
        if d2[k] is None:
            continue
        if k in d1:
            if d1[k] is None:
                continue
            d1[k] = torch.cat((d1[k], d2[k]), dim=0)
        else:
            d1[k] = d2[k]
    return d1

def prod(x):
    p = 1
    for i in x:
        p = p*i
    return p

def get_nullspace_projection(J, v):
    y_hat = torch.sum(J*v, -1, keepdim=True)/torch.sum(J * J, -1,  keepdim=True)
    x = v - (J * y_hat)
    # computing othogonal projection for pert with positive inner product as well
    return x

def measure_width(img, model, normalizer, index, dir_vec, distances = [5, 10, 15, 20, 25, 30], num_vecs=256):
    # print('Entered measure_width')
    softmax = torch.nn.Softmax(dim=-1)
    init_confidence = softmax(model(normalizer(img.clamp(0,1))))[:, index]
    rand_vecs = torch.randn(num_vecs, *img.shape[1:]).to(init_confidence.device)
    if torch.linalg.vector_norm(dir_vec) == 0:
        return np.zeros((len(distances)+1, 2))
    dir_vec = dir_vec/torch.linalg.vector_norm(dir_vec)
    a = torch.sum(rand_vecs*dir_vec, dim=(1,2,3))
    rand_vecs = rand_vecs - torch.einsum("ij, jklm -> iklm" ,a.unsqueeze(-1), dir_vec)
    rand_vecs = rand_vecs/torch.linalg.vector_norm(rand_vecs, dim=(1,2,3), keepdim=True)
    confs = [torch.Tensor((float(init_confidence), float(init_confidence)))]
    for i in distances:
        # print(i)
        pert_imgs = img + i*rand_vecs
        with torch.no_grad():
            pert_confs = softmax(model(normalizer(pert_imgs.clamp(0,1))))[:, index]
#         min_confs, max_confs = float(torch.min(pert_confs)), float(torch.max(pert_confs))
        confs.append(torch.quantile(pert_confs, q=torch.Tensor([0.05, 0.95]).to(pert_confs.device) ).cpu())
    confs = torch.stack(confs)
    return confs

def plot_images(images):
    with torch.no_grad():
        preds = torch.softmax(model(normalizer(images.to(device))), dim=-1).cpu().detach().numpy()
        predicted_classes = np.argmax(preds, axis=1)
        confidence_levels = np.max(preds, axis=1)
        # print(predicted_classes)
        # print(confidence_levels)
        
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for i in range(5):
        axes[i].imshow(images[i].permute(1, 2, 0).cpu().detach().numpy())  # Detach tensor before converting to numpy array
        axes[i].set_title(classes[predicted_classes[i]], fontsize=10)  # Set title as class name
        axes[i].set_xlabel(confidence_levels[i], fontsize=8, labelpad=5, color='blue', ha='center')

    plt.show()


In [156]:
def batch_level_set_traversal(model, normalizer, dataset, source_classes, target_image, target_class, stepsize, iterations, device, 
                           pthresh=0.05, inp=None, get_widths=False, get_confs_over_path=False, get_images=False, get_final_imgs=False, log_step=10, dfunc_list=[]):
    source_classes = source_classes.to(device)
    softmax = torch.nn.Softmax(dim=-1)
    if inp is None:
        inp = torch.Tensor(dataset).to(device)#.requires_grad_(True)
    else:
        inp = inp.detach().clone().to(device)#.requires_grad_(True)
    mask = torch.ones(*inp.shape, dtype=torch.bool).to(device)
    target_image = target_image.to(device)
    indices = torch.arange(len(inp))
    with torch.no_grad():
        pred = model(normalizer(inp.clamp(0,1)))
    init_probs = softmax(pred)
    # print(source_classes, init_probs[indices, source_classes])
    init_labels = torch.argmax(init_probs, dim=-1)
    rel_img_mask = (init_labels == source_classes).cpu()
    # all_dataset = dataset
    # all_source_classes = source_classes
    dataset = dataset[rel_img_mask]
    source_classes = source_classes[rel_img_mask]
    inp = inp[rel_img_mask]
    init_probs = init_probs[rel_img_mask]
    indices = torch.arange(len(inp))
    target_image = target_image[rel_img_mask]

    widths_over_path, inp_over_path, confs_over_path, confs_over_path_target = [], [], [], []

    adv_delta = None
    adv_step_size = 2e-3
    width_distances = [0.5, 1, 1.5, 2]
    adv_pert = True
    all_generated_inputs = []
    all_target_classes = []
    all_source_classes = []

    init_target_layer_acts = source_classes

    loss = torch.nn.CrossEntropyLoss(reduction='sum')
    inp = inp + (4/255)*(torch.rand_like(inp)-0.5)
    inp = torch.clamp(inp, 0., 1.)
    inp.requires_grad_()
    print('Started')
    for i in range(iterations):
        pred = model(normalizer(inp))
        probs = softmax(pred)
        target_activations = pred
        cost = loss(target_activations, init_target_layer_acts)
        cost.backward()
        J = inp.grad.reshape(len(probs), prod(inp.shape[1:]))
        v = target_image.flatten(start_dim=1) - inp.flatten(start_dim=1)
        # v = target_image.flatten() - inp.flatten(start_dim=1)
        null_vec = get_nullspace_projection(J, v).reshape(inp.shape)

        if adv_delta is None:
            adv_delta = (adv_step_size/2) * J
        else:
            adv_delta = adv_delta + (adv_step_size/2) * J
        adv_delta = torch.clamp(adv_delta, -adv_step_size, adv_step_size)
                
        if i%log_step == 0:
            # print(f'Iteration {i}')
            if get_widths:
                # print(f'Getting widths at {i}')
                with torch.no_grad():
                    widths_at_i = []
                    for j in range(len(inp)):
                        if probs[j, source_classes[j]] < pthresh:
                            widths_at_i.append(np.zeros((len(width_distances)+1,2)))
                        else:
                            widths_at_i.append(measure_width(inp[j:j+1], model, normalizer, source_classes[j], null_vec[j:j+1], distances=width_distances))
                    widths_over_path.append(torch.stack(widths_at_i))
            if get_images:
                with torch.no_grad():
                    inp_over_path.append(inp.detach().cpu())

            if get_confs_over_path:
                confs_at_i = probs[indices, source_classes].detach().cpu()
                confs_at_i_target = probs[indices, target_class].detach().cpu()
                confs_over_path.append(confs_at_i)
                confs_over_path_target.append(confs_at_i_target)

        if adv_pert:
            new_inp = inp + stepsize*null_vec - adv_delta.reshape(inp.shape)
        else:
            new_inp = inp + stepsize*null_vec
        new_inp = torch.clamp(new_inp, 0.0, 1.0)
        
        with torch.no_grad():
            pred_probs = softmax(model(normalizer(new_inp)))

        mask = ((init_probs[indices,source_classes] - pred_probs[indices,source_classes] < pthresh)).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
        # if ~ (mask.any()):
        #     print("Iterations stopped at:", i)
        #     break
        inp = (inp*(~mask) + new_inp*mask).detach()
        inp.requires_grad_()
        # if(i%10 == 0):
        #     #Plots
        #     plot_images(inp)
        # Store generated inputs and target classes
        if i==0:
            for k in range(6):
                all_generated_inputs.append(inp.clone().detach())
                all_target_classes.append(source_classes.clone().detach())
                all_source_classes.append(source_classes.clone().detach())

        if i>340:
            all_generated_inputs.append(inp.clone().detach())
            all_target_classes.append(target_class.clone().detach())
            all_source_classes.append(source_classes.clone().detach())
    
    return all_generated_inputs, all_target_classes, all_source_classes

In [157]:
example_loader = [(target_images,  torch.LongTensor(target_classes))]
iterations = 350
all_generated_inputs1 = []
all_target_classes1 = []
all_source_classes1 = []
for i, image in enumerate(target_images):
    print(i)
    accum_dict = dict([])
    for b, data_batch in enumerate(tqdm(example_loader)):
        # print(b) 0
        batch_target_classes = torch.Tensor([target_classes[i]]*len(data_batch[0])).long()   #[i, i, i, i, i]
        batch_target_images = torch.stack([image]*len(data_batch[0]), dim=0)                 #[[image]*5]
        generated_inputs, target_classes_all, source_classes_all  = batch_level_set_traversal(model, normalizer, data_batch[0], data_batch[1],
                                                    batch_target_images, batch_target_classes, stepsize=1e-2, iterations=iterations, device=device, 
                                                    pthresh=0.1, get_widths=True, get_images=True, get_final_imgs=True,
                                                    dfunc_list=[], get_confs_over_path=True, log_step=50)
        all_generated_inputs1.append(generated_inputs)
        all_target_classes1.append(target_classes_all)
        all_source_classes1.append(source_classes_all)

all_generated_inputs1 = [element for sublist in all_generated_inputs1 for element in sublist]
all_target_classes1 = [element for sublist in all_target_classes1 for element in sublist]
all_source_classes1 = [element for sublist in all_source_classes1 for element in sublist]

0


  0%|          | 0/1 [00:00<?, ?it/s]

Started


100%|██████████| 1/1 [00:37<00:00, 37.22s/it]


1


...

100%|██████████| 1/1 [00:36<00:00, 36.94s/it]


9


  0%|          | 0/1 [00:00<?, ?it/s]

Started


100%|██████████| 1/1 [00:36<00:00, 36.88s/it]


In [47]:
# model, normalizer = load_model_normalizer(2)

# example_loader = [(target_images,  torch.LongTensor(target_classes))]
# iterations = 500
# all_generated_inputs2 = []
# all_target_classes2 = []
# all_source_classes2 = []
# for i, image in enumerate(target_images):
#     print(i)
#     accum_dict = dict([])
#     for b, data_batch in enumerate(tqdm(example_loader)):
#         # print(b) 0
#         batch_target_classes = torch.Tensor([target_classes[i]]*len(data_batch[0])).long()   #[i, i, i, i, i]
#         batch_target_images = torch.stack([image]*len(data_batch[0]), dim=0)                 #[[image]*5]
#         generated_inputs, target_classes_all, source_classes_all  = batch_level_set_traversal(model, normalizer, data_batch[0], data_batch[1],
#                                                     batch_target_images, batch_target_classes, stepsize=1e-2, iterations=iterations, device=device, 
#                                                     pthresh=0.1, get_widths=True, get_images=True, get_final_imgs=True,
#                                                     dfunc_list=[], get_confs_over_path=True, log_step=50)
#         all_generated_inputs2.append(generated_inputs)
#         all_target_classes2.append(target_classes_all)
#         all_source_classes2.append(source_classes_all)

# all_generated_inputs2 = [element for sublist in all_generated_inputs2 for element in sublist]
# all_target_classes2 = [element for sublist in all_target_classes2 for element in sublist]
# all_source_classes2 = [element for sublist in all_source_classes2 for element in sublist]

Downloading...
From (original): https://drive.google.com/uc?id=1-WJWpAZLlmc4gJ8XXNf7IETjnSZzaCNp
From (redirected): https://drive.google.com/uc?id=1-WJWpAZLlmc4gJ8XXNf7IETjnSZzaCNp&confirm=t&uuid=e5335381-d52c-4851-b2ab-c62f95ba36c8
To: /home/iit/Downloads/BlindSpots Geometry/models/cifar10/Linf/Wu2020Adversarial_extra.pt
100%|██████████| 153M/153M [02:23<00:00, 1.07MB/s] 


0


  0%|          | 0/1 [00:00<?, ?it/s]

Started


100%|██████████| 1/1 [01:03<00:00, 63.50s/it]


...

5


  0%|          | 0/1 [00:00<?, ?it/s]

Started


  0%|          | 0/1 [00:39<?, ?it/s]


KeyboardInterrupt: 

## Adversarial Input Data

In [25]:
# all_generated_inputs = all_generated_inputs1 + all_generated_inputs2
# all_target_classes = all_target_classes1 + all_target_classes2
# all_source_classes = all_source_classes1 + all_source_classes2
# all_generated_inputs = all_generated_inputs1
# all_target_classes = all_target_classes1
# all_source_classes = all_source_classes1
# print(len(all_generated_inputs))

# file_path = "all_generated_inputs.pkl"
# with open(file_path, 'wb') as file:
#     pickle.dump(all_generated_inputs, file)

# file_path2 = "all_target_classes.pkl"
# with open(file_path2, 'wb') as file:
#     pickle.dump(all_target_classes, file)

340


In [162]:
## load data
file_path = "all_generated_inputs.pkl"
with open(file_path, 'rb') as file:
    all_generated_inputs = pickle.load(file)

file_path2 = "all_target_classes.pkl"
with open(file_path2, 'rb') as file:
    all_target_classes = pickle.load(file)

# #update data
# all_generated_inputs = all_generated_inputs + all_generated_inputs1
# all_target_classes = all_target_classes + all_target_classes1

# #save data
# file_path = "all_generated_inputs.pkl"
# with open(file_path, 'wb') as file:
#     pickle.dump(all_generated_inputs, file)

# file_path2 = "all_target_classes.pkl"
# with open(file_path2, 'wb') as file:
#     pickle.dump(all_target_classes, file)

print(all_generated_inputs[0].shape)
print(len(all_target_classes))



adversarial_inputs = torch.cat(all_generated_inputs, dim=0)
target_class = torch.cat(all_target_classes, dim=0)
print(adversarial_inputs.shape)

torch.Size([10, 3, 32, 32])
1890
torch.Size([18900, 3, 32, 32])


In [32]:
import torch
from torchvision import transforms
from PIL import Image
import random

# Set the seed for reproducibility
torch.manual_seed(42)
random.seed(42)

# Define the data augmentation transformations
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    transforms.ToTensor()
])

# Randomly select 10,000 indices
random_indices = random.sample(range(20000), 12000)

# Apply data augmentation to the randomly selected images
for idx in random_indices:
    img = adversarial_inputs[idx].permute(1, 2, 0).cpu().numpy()  # Convert to numpy array
    img = Image.fromarray((img * 255).astype('uint8'))  # Convert to PIL image
    img = data_transforms(img)  # Apply transformations
    adversarial_inputs[idx] = img

print(adversarial_inputs.shape)

torch.Size([25950, 3, 32, 32])


## Training On Adversarial Inputs

In [164]:
def adversarial_training(model, optimizer, criterion, adversarial_inputs, target_class, num_epochs=10, batch_size=196):
    model.train()
    # adversarial_inputs = torch.cat(adversarial_inputs, dim=0)
    # print(adversarial_inputs.shape)
    # target_class = torch.cat(target_class, dim=0)
    adversarial_inputs.to(device)
    target_class.to(device)

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
    subset_indices = np.random.choice(len(train_dataset), 8000, replace=False)
    train_subset = torch.utils.data.Subset(train_dataset, subset_indices)
    subset_dataloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    dataset = torch.utils.data.TensorDataset(adversarial_inputs, target_class)
    # combined_dataset = torch.utils.data.ConcatDataset([train_subset, dataset])

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # adversarial_inputs.requires_grad_()
    i = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            inputs.requires_grad = True

            optimizer.zero_grad()
            outputs = model(inputs)
            if i==0:
                outputs_cpu = outputs.cpu().detach()

                # Convert to numpy arrays
                predicted_classes = np.argmax(outputs_cpu.numpy(), axis=1)
                outputs_softmax = F.softmax(outputs, dim=1)

                # Convert to numpy arrays
                confidence_levels = np.max(outputs_softmax.cpu().detach().numpy(), axis=1)
                print("Pred")
                print(predicted_classes)
                print("conf")
                print(confidence_levels)
                print("tar")
                print(targets)
            i= i+1
            loss = criterion(outputs, targets)
            loss = loss * 50
            loss.backward()
            optimizer.step()

            # Compute statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        for inputs, targets in subset_dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            inputs.requires_grad = True

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Compute statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        

        # Print statistics
        epoch_loss = running_loss / total
        epoch_accuracy = correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    print('Finished Training')


In [165]:
model, normalizer = load_model_normalizer(2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

adversarial_training(model, optimizer, criterion, adversarial_inputs, target_class, num_epochs=500, batch_size=128)

Files already downloaded and verified
Pred
[6 0 7 5 3 6 3 2 0 1 7 0 2 3 9 1 3 7 0 6 3 9 0 8 1 5 2 3 7 0 3 7 1 4 2 3 1
 5 4 3 1 0 0 2 0 3 0 9 3 3 5 6 1 3 9 1 1 1 7 7 2 1 9 1 5 2 1 2 1 9 2 4 3 5
 1 1 1 2 5 2 0 4 4 2 9 6 0 9 6 3 6 5 0 6 9 4 5 4 7 7 6 3 1 4 7 2 7 2 9 5 2
 3 1 3 1 3 3 9 3 3 2 4 5 6 9 2 3 1]
conf
[0.38545558 0.5421197  0.54871553 0.33548337 0.28940564 0.508948
 0.75017405 0.63591975 0.7326899  0.8823934  0.93732554 0.73833203
 0.6059256  0.41117257 0.7834212  0.74626833 0.25208437 0.5485015
 0.94066167 0.70484376 0.26739565 0.77859145 0.55133086 0.9496612
 0.9332201  0.34038055 0.7084483  0.25434452 0.93810016 0.54436606
 0.50304186 0.9378164  0.8760861  0.6559403  0.4043861  0.4839951
 0.8772843  0.33773577 0.8881931  0.2547413  0.7377586  0.7387417
 0.93525916 0.696376   0.94163644 0.24515855 0.5409731  0.6445433
 0.47075462 0.25889808 0.3538518  0.38647583 0.74493724 0.707259
 0.7352008  0.88248116 0.7964431  0.76511985 0.7366374  0.89878666
 0.63691527 0.80279285 0.65222

KeyboardInterrupt: 

In [36]:
torch.save(model.state_dict(), 'Wu2020Adversarial_extra_trained.pt')